In [ ]:
import pandas
import matplotlib
import pandas as pd
import numpy as np

matplotlib.use("TkAgg")
pandas.set_option('display.max_columns', None)

In [ ]:
path = 'pp-2021.csv'
data = pandas.read_csv(path, sep=',', skiprows=1)

In [ ]:
data = data.drop(data.columns[[0, 2, 7, 8, 9, 10, 12, 13, 14, 15]], axis=1)
data.columns = ["Price", "Postcode", "PropertyType", "OldNew", "Duration", "City"]

In [ ]:
# Remove rows that describe properties that are not typically Houses
data = data[~data["PropertyType"].isin(["F", "O"])]

In [ ]:
# For cities that appear in the data less than 600 times, join into "Other" category
city_stats = data.groupby('City')['City'].agg('count').sort_values(ascending=False)
city_stats_less_than_600 = city_stats[city_stats <= 600]
data["City"] = data["City"].apply(lambda x: 'Other' if x in city_stats_less_than_600 else x)
# Print city count distribution
city_stats

In [ ]:
# Remove rows with prices lower than £40,000 and higher than £5,000,000
data = data[data["Price"] > 40000]
data = data[data["Price"] < 5000000]
data.Price.describe()

In [ ]:
# Convert PropertyType and Duration from string to int
data['PropertyType'] = [ord(x) - 64 for x in data.PropertyType]
data['Duration'] = [ord(x) - 64 for x in data.Duration]
# Convert OldNew from Y/N to 1/0
data['OldNew'] = data['OldNew'].map({'Y': 1, 'N': 0})

In [ ]:
# Import/Install pgeocode lib to get longitude and latitude from a postcode string
import sys
try:
    import pgeocode
except ImportError as e:
    !conda install --channel conda-forge --yes --prefix {sys.prefix} pgeocode

In [ ]:
# Remove all rows with NaN
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

# Format postcode data in main df so that it matches the format in the pgeocode lib
codes = [str(x) for x in data.Postcode]
codes = [x[0:x.index(" ")] for x in codes if " " in x]
data

In [ ]:
# Get longitude and latitude values for every valid postcode
nomi = pgeocode.Nominatim("GB")
post_code_info = nomi.query_postal_code(codes)
latitude = post_code_info.latitude
longitude = post_code_info.longitude
post_code_info

In [ ]:
# Store long and lat in main df
data['Longitude'] = longitude
data['Latitude'] = latitude

# Drop Postcode col now we have location data in long + lat
data.drop(['Postcode'], inplace=True, axis=1)

data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
# Convert each city into a column using one-hot encoding, remove old City col
dummies = pandas.get_dummies(data.City)
dummies.drop(('Other'), inplace=True, axis=1)
data = pd.concat([data, dummies], axis=1)
data.drop(['City'], inplace=True, axis=1)

In [ ]:
# Normalize Price between -1 and 1
min_params = data['Price'].min()
max_params = data['Price'].max()
data['Price'] = 2 * ((data['Price'] - min_params) / (max_params - min_params)) -1

# Normalize lat and longs between~ -1 and 1
data[['Longitude', 'Latitude']] = (data[['Longitude', 'Latitude']] / 100)

In [ ]:
# Training, Validation, Test split = 80%, 10%, 10%
train, val, test = np.split(data.sample(frac=1, random_state=54), [int(.8 * len(data)), int(.9 * len(data))])

In [ ]:
# y = column vector, true price value (labels) (len(data[0],)
y_train, y_val, y_test = train.Price, val.Price, test.Price
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
# X = matrix, excludes price feature
train.drop(['Price'], axis=1, inplace=True)
X_train = train.to_numpy()
val.drop(['Price'], axis=1, inplace=True)
X_val = val.to_numpy()
test.drop(['Price'], axis=1, inplace=True)
X_test = test.to_numpy()

In [186]:
data.head()

,Price,PropertyType,OldNew,Duration,Longitude,Latitude,ABERDARE,ABINGDON,ACCRINGTON,ALDERSHOT,ALFRETON,ALTRINCHAM,ANDOVER,ASHFORD,ASHTON-UNDER-LYNE,AYLESBURY,BANBURY,BARNET,BARNSLEY,BARNSTAPLE,BARROW-IN-FURNESS,BARRY,BASILDON,BASINGSTOKE,BATH,BEDFORD,BELPER,BENFLEET,BEVERLEY,BEXHILL-ON-SEA,BEXLEYHEATH,BICESTER,BIDEFORD,BILLERICAY,BILLINGHAM,BILSTON,BIRKENHEAD,BIRMINGHAM,BISHOP AUCKLAND,BISHOP'S STORTFORD,BLACKBURN,BLACKPOOL,BLYTH,BOGNOR REGIS,BOLTON,BOOTLE,BOSTON,BOURNEMOUTH,BRACKNELL,BRADFORD,BRAINTREE,BRENTWOOD,BRIDGEND,BRIDGWATER,BRIDLINGTON,BRIGHTON,BRISTOL,BROMLEY,BROMSGROVE,BUCKINGHAM,BURGESS HILL,BURNLEY,BURTON-ON-TRENT,BURY,BURY ST EDMUNDS,CAERPHILLY,CAMBERLEY,CAMBRIDGE,CANNOCK,CANTERBURY,CANVEY ISLAND,CARDIFF,CARLISLE,CARMARTHEN,CASTLEFORD,CHATHAM,CHEADLE,CHELMSFORD,CHELTENHAM,CHESTER,CHESTER LE STREET,CHESTERFIELD,CHICHESTER,CHIPPENHAM,CHORLEY,CHRISTCHURCH,CIRENCESTER,CLACTON-ON-SEA,CLEETHORPES,CLITHEROE,COALVILLE,COLCHESTER,CONGLETON,CONSETT,CORBY,COVENTRY,CRAWLEY,CREWE,CROYDON,CWMBRAN,DAGENHAM,DARLINGTON,DARTFORD,DARWEN,DAVENTRY,DEAL,DEESIDE,DERBY,DEREHAM,DEVIZES,DEWSBURY,DIDCOT,DONCASTER,DORCHESTER,DOVER,DRIFFIELD,DUDLEY,DUNSTABLE,DURHAM,EASTBOURNE,EASTLEIGH,ELLESMERE PORT,ELY,ENFIELD,EPSOM,EVESHAM,EXETER,EXMOUTH,FAREHAM,FARNBOROUGH,FARNHAM,FLEET,FOLKESTONE,FROME,GAINSBOROUGH,GATESHEAD,GILLINGHAM,GLOUCESTER,GODALMING,GOOLE,GOSPORT,GRANTHAM,GRAVESEND,GRAYS,GREAT YARMOUTH,GRIMSBY,GUILDFORD,HAILSHAM,HALESOWEN,HALIFAX,HARLOW,HARROGATE,HARROW,HARTLEPOOL,HASTINGS,HAVANT,HAVERFORDWEST,HAYES,HAYWARDS HEATH,HEMEL HEMPSTEAD,HEREFORD,HERNE BAY,HIGH WYCOMBE,HINCKLEY,HITCHIN,HORNCHURCH,HORSHAM,HOUGHTON LE SPRING,HOVE,HUDDERSFIELD,HULL,HUNTINGDON,HYDE,ILFORD,ILKESTON,ILKLEY,IPSWICH,KEIGHLEY,KENDAL,KETTERING,KIDDERMINSTER,KING'S LYNN,KINGSWINFORD,LANCASTER,LEAMINGTON SPA,LEATHERHEAD,LEEDS,LEICESTER,LEIGH,LEIGH-ON-SEA,LEIGHTON BUZZARD,LEYLAND,LICHFIELD,LINCOLN,LITTLEHAMPTON,LIVERPOOL,LLANELLI,LONDON,LOUGHBOROUGH,LOUTH,LOWESTOFT,LUTON,LYMINGTON,LYTHAM ST ANNES,MACCLESFIELD,MAIDENHEAD,MAIDSTONE,MALDON,MALVERN,MANCHESTER,MANSFIELD,MARCH,MARGATE,MARKET HARBOROUGH,MATLOCK,MELTON MOWBRAY,MERTHYR TYDFIL,MIDDLESBROUGH,MILTON KEYNES,MORECAMBE,MORPETH,NANTWICH,NEATH,NELSON,NEWARK,NEWBURY,NEWCASTLE,NEWCASTLE UPON TYNE,NEWPORT,NEWTON ABBOT,NORTH SHIELDS,NORTHAMPTON,NORTHWICH,NORWICH,NOTTINGHAM,NUNEATON,OLDBURY,OLDHAM,ORMSKIRK,ORPINGTON,OSWESTRY,OXFORD,PAIGNTON,PENRITH,PENZANCE,PETERBOROUGH,PETERLEE,PLYMOUTH,PONTEFRACT,PONTYPOOL,PONTYPRIDD,POOLE,PORT TALBOT,PORTSMOUTH,POULTON-LE-FYLDE,PRESCOT,PRESTON,PUDSEY,RAMSGATE,RAYLEIGH,READING,REDCAR,REDDITCH,REDHILL,RETFORD,RHYL,RICHMOND,ROCHDALE,ROCHESTER,ROMFORD,ROSSENDALE,ROTHERHAM,ROYSTON,RUGBY,RUGELEY,RUISLIP,RUNCORN,RUSHDEN,SALE,SALFORD,SALISBURY,SANDBACH,SCARBOROUGH,SCUNTHORPE,SELBY,SEVENOAKS,SHEERNESS,SHEFFIELD,SHIPLEY,SHREWSBURY,SIDCUP,SITTINGBOURNE,SKEGNESS,SKELMERSDALE,SLEAFORD,SLOUGH,SOLIHULL,SOUTH CROYDON,SOUTH SHIELDS,SOUTHAMPTON,SOUTHEND-ON-SEA,SOUTHPORT,SOUTHSEA,SPALDING,ST ALBANS,ST AUSTELL,ST HELENS,ST IVES,ST LEONARDS-ON-SEA,ST NEOTS,STAFFORD,STAMFORD,STANFORD-LE-HOPE,STANLEY,STEVENAGE,STOCKPORT,STOCKTON-ON-TEES,STOKE-ON-TRENT,STOURBRIDGE,STOWMARKET,STRATFORD-UPON-AVON,STROUD,SUDBURY,SUNDERLAND,SUTTON,SUTTON COLDFIELD,SUTTON-IN-ASHFIELD,SWADLINCOTE,SWANSEA,SWINDON,TAMWORTH,TAUNTON,TELFORD,THETFORD,THORNTON-CLEVELEYS,TONBRIDGE,TORQUAY,TROWBRIDGE,TRURO,TUNBRIDGE WELLS,TWICKENHAM,UXBRIDGE,WAKEFIELD,WALLASEY,WALLSEND,WALSALL,WALTHAM CROSS,WALTON-ON-THAMES,WARRINGTON,WARWICK,WASHINGTON,WATERLOOVILLE,WATFORD,WELLINGBOROUGH,WESTON-SUPER-MARE,WEYMOUTH,WHITLEY BAY,WHITSTABLE,WICKFORD,WIDNES,WIGAN,WIGSTON,WILMSLOW,WIMBORNE,WINCHESTER,WINDSOR,WIRRAL,WISBECH,WITNEY,WOKING,WOKINGHAM,WOLVERHAMPTON,WOODBRIDGE,WORCESTER,WORKSOP,WORTHING,WREXHAM,YEOVIL,YORK
0,-0.960927,20,0,6,-0.018961,0.523238,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
# Shuffle order of indexes for training data
np.set_printoptions(threshold=0)
keys = np.array(range(len(y_train)))
np.random.shuffle(keys)
X_train = X_train[keys]
y_train = y_train[keys]

In [ ]:
import numpy as np

class Dense_Layer:
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0., weight_regularizer_l2=0.,
                 bias_regularizer_l1=0., bias_regularizer_l2=0.):

        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        self.inputs = inputs

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1

        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights

        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1

        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        self.dinputs = np.dot(dvalues, self.weights.T)


In [ ]:
class Dropout_Layer:
    def __init__(self, rate):
        self.rate = 1 - rate

    def forward(self, inputs):
        self.inputs = inputs

        self.binary_mask = np.random.binomial(1, self.rate, size=inputs.shape) / self.rate
        self.output = inputs * self.binary_mask

    def backward(self, dvalues):
        self.dinputs = dvalues * self.binary_mask

In [ ]:
class Linear_Activation:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

In [ ]:
class ReLU_Activation:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

In [ ]:
class Loss:
    def calculate(self, output, y, layers, include_regularization=False):
        sample_losses = self.forward(output, y, layers)
        data_loss = np.mean(sample_losses)


        self.accumulated_sum += np.sum(sample_losses)
        self.accumulated_count += len(sample_losses)

        if not include_regularization:
            return data_loss

        return data_loss, self.regularization_loss(layers)

    def calculate_accumulated(self, include_regularization=False):
        data_loss = self.accumulated_sum / self.accumulated_count

        if not include_regularization:
            return data_loss


        return data_loss, self.regularization_loss(layers)

    def regularization_loss(self, layers_list):

        regularization_loss = 0

        for layer in layers_list:
            if layer.weight_regularizer_l1 > 0:
                regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))

            if layer.weight_regularizer_l2 > 0:
                regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)

            if layer.bias_regularizer_l1 > 0:
                regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

            if layer.bias_regularizer_l2 > 0:
                regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)

        return regularization_loss

    def new_pass(self):
        self.accumulated_sum = 0
        self.accumulated_count = 0

In [ ]:
class MSE_Loss(Loss):
    def forward(self, y, y_hat, layers):
        sample_losses = np.mean((y_hat - y)**2, axis=-1)
        return sample_losses
    def backward(self, dvalues, y_hat):

        samples = len(dvalues)
        for batch in y_hat:
            self.dinputs = (-2 * (batch - dvalues)) / samples


In [ ]:
class SGD_Optimzer:
    def __init__(self, learning_rate = 0.01, decay=1e-6, momentum=0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1 / (1 + self.decay * self.iterations))
    def update_params(self, layer):

        if not hasattr(layer, 'weight_momentums'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)

            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        else:
            weight_updates = -self.learning_rate * layer.dweights
            bias_updates = -self.learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1

In [ ]:
class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [ ]:
class Accuracy:
    def __init__(self):
        pass

    def calculate(self, predictions, y):
        # Get comparison results
        comparisons = self.compare(predictions, y)
        # Calculate an accuracy
        accuracy = np.mean(comparisons)
        # Add accumulated sum of matching values and sample count
        self.accumulated_sum += np.sum(comparisons)
        self.accumulated_count += len(comparisons)
        # Return accuracy
        return accuracy

    def calculate_accumulated(self):
        # Calculate an accuracy
        accuracy = self.accumulated_sum / self.accumulated_count
        # Return the data and regularization losses
        return accuracy
        # Reset variables for accumulated accuracy

    def compare(self, predictions, y):
        return np.absolute(predictions - y)

    def new_pass(self):
        self.accumulated_sum = 0
        self.accumulated_count = 0


In [180]:
dense1 = Dense_Layer(X_train.shape[1], 128, weight_regularizer_l1=1e-4, bias_regularizer_l1=1e-4)
activation1 = ReLU_Activation()
dropout1 = Dropout_Layer(0.1)
dense2 = Dense_Layer(128, 128)
activation2 = ReLU_Activation()
dropout2 = Dropout_Layer(0.1)
dense3 = Dense_Layer(128, 1)
activation3 = Linear_Activation()
loss_function = MSE_Loss()
#optimizer = SGD_Optimzer(learning_rate=0.01, decay=4e-5)
optimizer = Optimizer_Adam(learning_rate=0.01, decay=1e-3)
batch_accuracy = Accuracy()



layers = [dense1, dense2, dense3]

In [176]:
batch_size = 64
train_steps = X_train.shape[0] // batch_size
val_steps = X_val.shape[0] // batch_size
test_steps = X_val.shape[0] // 512

if train_steps * batch_size < X_train.shape[0]:
    train_steps += 1

if val_steps * batch_size < X_val.shape[0]:
    val_steps += 1

if test_steps * 512 < X_test.shape[0]:
    test_steps += 1



In [181]:
# Begin training model
for epoch in range(10):
    print(f'epoch: {epoch}')
    np.random.shuffle(keys)
    X_train = X_train[keys]
    y_train = y_train[keys]
    loss_function.new_pass()
    batch_accuracy.new_pass()

    # Train model in mini batches
    for step in range(train_steps):
        batch_X = X_train[step * batch_size:(step+1)*batch_size]
        batch_y = y_train[step * batch_size:(step+1)*batch_size]

        # Forward Pass
        dense1.forward(batch_X)
        activation1.forward(dense1.output)
        dropout1.forward(activation1.output)
        dense2.forward(dropout1.output)
        activation2.forward(dense2.output)
        dropout2.forward(activation2.output)
        dense3.forward(dropout2.output)
        activation3.forward(dense3.output)

        # Calculate Loss of current batch
        data_loss, reg_loss = loss_function.calculate(activation3.output, batch_y, layers, include_regularization=True)
        loss = data_loss + reg_loss

        # Predicted Prices of current batch
        predictions = activation3.output

        # Backward Pass
        loss_function.backward(activation3.output, batch_y)
        activation3.backward(loss_function.dinputs)
        dense3.backward(activation3.dinputs)
        dropout2.backward(dense3.dinputs)
        activation2.backward(dropout2.dinputs)
        dense2.backward(activation2.dinputs)
        dropout1.backward(dense2.dinputs)
        activation1.backward(dropout1.dinputs)
        dense1.backward(activation1.dinputs)

        # Update weights
        optimizer.pre_update_params()
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.update_params(dense3)
        optimizer.post_update_params()

        # Print loss every 1000 batches
        if not step % 1000:
            print(f'batch: {step} out of {train_steps}, loss: {loss:.3f}')

    # Calculate mean Loss after epoch
    epoch_data_loss, epoch_reg_loss = \
        loss_function.calculate_accumulated(include_regularization=True)
    epoch_loss = epoch_data_loss + epoch_reg_loss

    # Print training progress after epoch
    print(f'training, ' +
    f'loss: {epoch_loss:.3f} (' +
    f'data_loss: {epoch_data_loss:.3f}, ' +
    f'reg_loss: {epoch_reg_loss:.3f}, ' +
    f'lr: {optimizer.current_learning_rate}')

    # Reset mean Loss before using validation set
    loss_function.new_pass()
    batch_accuracy.new_pass()

    # Pass through validation set for evaluation during training
    for step in range(val_steps):

        batch_X = X_val[step * batch_size:(step+1)*batch_size]
        batch_y = y_val[step * batch_size:(step+1)*batch_size]

        dense1.forward(batch_X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)
        dense3.forward(activation2.output)
        activation3.forward(dense3.output)

        loss_function.calculate(activation3.output, batch_y, layers)

        predictions = activation3.output

    # Calculate mean Loss for validation set on current model parameters
    val_loss = loss_function.calculate_accumulated()

    # Print mean validation loss per epoch
    print(f'validation, ' +
          f'loss: {val_loss:.3f}, ')

# Training finished

# Test model
print("Model Evaluation on Test Set")

loss_function.new_pass()
batch_accuracy.new_pass()

# Pass through testing set
for step in range(test_steps):

    batch_X = X_test[step * 512:(step+1)*512]
    batch_y = y_test[step * 512:(step+1)*512]

    dense1.forward(batch_X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    loss_function.calculate(activation3.output, batch_y, layers)

    predictions = activation3.output

# Calculate mean Loss for validation set on current model parameters
test_loss = loss_function.calculate_accumulated()

# Print mean validation loss per epoch
print(f'test set, ' +
      f'loss: {test_loss:.3f}, ')

epoch: 0
batch: 0 out of 10802, loss: 2.087
batch: 1000 out of 10802, loss: 0.016
batch: 2000 out of 10802, loss: 0.030
batch: 3000 out of 10802, loss: 0.012
batch: 4000 out of 10802, loss: 0.011
batch: 5000 out of 10802, loss: 0.020
batch: 6000 out of 10802, loss: 0.010
batch: 7000 out of 10802, loss: 0.007
batch: 8000 out of 10802, loss: 0.006
batch: 9000 out of 10802, loss: 0.006
batch: 10000 out of 10802, loss: 0.010
training, loss: 0.022 (data_loss: 0.021, reg_loss: 0.001, lr: 0.0008473858147614609
validation, loss: 0.014, 
epoch: 1
batch: 0 out of 10802, loss: 0.008
batch: 1000 out of 10802, loss: 0.008
batch: 2000 out of 10802, loss: 0.049
batch: 3000 out of 10802, loss: 0.006
batch: 4000 out of 10802, loss: 0.014
batch: 5000 out of 10802, loss: 0.010
batch: 6000 out of 10802, loss: 0.010
batch: 7000 out of 10802, loss: 0.032
batch: 8000 out of 10802, loss: 0.006
batch: 9000 out of 10802, loss: 0.013
batch: 10000 out of 10802, loss: 0.031
training, loss: 0.014 (data_loss: 0.014,

In [ ]:
# Add accuracy
# Make some predictions, if predictions are good, test accuracy on external dataset